<a href="https://www.kaggle.com/code/ibkya12/map-0-97-leashmp-competition?scriptVersionId=180401033" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 48.3 MB/s eta 0:00:00


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 72.4 MB/s eta 0:00:00


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 200000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 200000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,256600004,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cl.NCC1CCOCC12CCCC2,CC(CN)S(C)=O,CC(CNc1nc(NCC2CCOCC23CCCC3)nc(N[C@H](CC(=O)N[D...,sEH,0
1,187916341,O=C(Nc1ccc(C(=O)O)cc1)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1nnc2n1CCOCC2,Cl.Cl.NCc1cncc(F)c1,O=C(N[Dy])c1ccc(Nc2nc(NCc3cncc(F)c3)nc(NCc3nnc...,HSA,0
2,267961744,O=C(O)C[C@H](Cc1ccc(Cl)c(Cl)c1)NC(=O)OCC1c2ccc...,Nc1nc(Cl)ccc1Cl,NCc1ccccc1N1CCOCC1,O=C(C[C@H](Cc1ccc(Cl)c(Cl)c1)Nc1nc(NCc2ccccc2N...,HSA,0
3,72866718,O=C(NC(CC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ccc2c(c1)CNCC2,Nc1cc(Cl)nnc1Cl,O=C(N[Dy])C(CC1CCCCC1)Nc1nc(Nc2ccc3c(c2)CNCC3)...,BRD4,0
4,266817610,O=C(O)C[C@H](Cc1ccc(C(F)(F)F)cc1)NC(=O)OCC1c2c...,Nc1cccc2c1C(=O)N(C1CCC(=O)NC1=O)C2=O,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,O=C(C[C@H](Cc1ccc(C(F)(F)F)cc1)Nc1nc(NCc2nc3c(...,HSA,0


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [6]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [7]:
from xgboost import XGBClassifier

In [8]:
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train , y_train)

y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.97


In [9]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.120958
0:	learn: 0.6510384	total: 209ms	remaining: 3m 28s
1:	learn: 0.6157022	total: 374ms	remaining: 3m 6s
2:	learn: 0.5919785	total: 509ms	remaining: 2m 49s
3:	learn: 0.5726544	total: 652ms	remaining: 2m 42s
4:	learn: 0.5566899	total: 790ms	remaining: 2m 37s
5:	learn: 0.5415104	total: 930ms	remaining: 2m 34s
6:	learn: 0.5299998	total: 1.11s	remaining: 2m 37s
7:	learn: 0.5203334	total: 1.26s	remaining: 2m 36s
8:	learn: 0.5103880	total: 1.38s	remaining: 2m 31s
9:	learn: 0.5023895	total: 1.51s	remaining: 2m 30s
10:	learn: 0.4966807	total: 1.64s	remaining: 2m 27s
11:	learn: 0.4912118	total: 1.78s	remaining: 2m 26s
12:	learn: 0.4858894	total: 1.91s	remaining: 2m 24s
13:	learn: 0.4800215	total: 2.06s	remaining: 2m 25s
14:	learn: 0.4748511	total: 2.19s	remaining: 2m 23s
15:	learn: 0.4694464	total: 2.32s	remaining: 2m 22s
16:	learn: 0.4653454	total: 2.45s	remaining: 2m 21s
17:	learn: 0.4613439	total: 2.59s	remaining: 2m 21s
18:	learn: 0.4575535	total: 2.71s	remaining: 

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [10]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = xgb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
